In [2]:
from shared.preface import *
import shared.functions as fct

# ------------------------------------- #
# Select halo to do gravity tests with. #
# ------------------------------------- #

sim = 'L012N376'
snap = '0036'
z = 0 
mass_gauge = 12.5  # in log10 Msun
mass_range = 0.3

hname = f'1e+{mass_gauge}_pm{mass_range}Msun_Test'
fct.halo_batch_indices(sim, snap, mass_gauge, mass_range, 'halos', hname)
halo_batch_IDs = np.load(f'{sim}/halo_batch_{hname}_indices.npy')
halo_batch_params = np.load(f'{sim}/halo_batch_{hname}_params.npy')
halo_num = len(halo_batch_params)

print('Halo batch params (Rvir,Mvir,cNFW):')
print(halo_batch_params)

halo_ID = halo_batch_IDs[1]
print(halo_ID)
halo_j = 1

halo_rvir = halo_batch_params[halo_j, 0]
halo_Mvir = halo_batch_params[halo_j, 1]
halo_cNFW = halo_batch_params[halo_j, 2]

halo_rvir *= kpc
halo_Mvir = 10**halo_Mvir * Msun
halo_rho0 = fct.scale_density_NFW(z, halo_cNFW)
halo_Rs = halo_rvir / halo_cNFW


# --------------------------- #
# Read and load DM positions. #
# --------------------------- #

IDname = f'origID{halo_ID}_snap_{snap}_Test'
fct.read_DM_halo_index(sim, snap, halo_ID, IDname)
DM_raw = np.load(f'{sim}/DM_pos_{IDname}.npy')  

print(len(DM_raw))

# ---------------------- #
# Cell division process. #
# ---------------------- #

# Initialize grid.
GRID_L = (int(np.abs(DM_raw).max()) + 1)*kpc
raw_grid = fct.grid_3D(GRID_L, GRID_L)  # to get 8 all-covering cells
init_grid = np.expand_dims(raw_grid, axis=1)

# Prepare arrays for cell division.
DM_raw *= kpc
DM_pos = np.expand_dims(DM_raw, axis=0)
DM_pos_for_cell_division = np.repeat(DM_pos, len(init_grid), axis=0)

# Cell division.
DM_lim = 100000
cell_division_count = fct.cell_division(
    init_grid, DM_pos_for_cell_division, GRID_L, DM_lim, None, 
    sim, IDname
)

# Load files from cell division.
fin_grid = np.load(f'{sim}/fin_grid_{IDname}.npy')
DM_count = np.load(f'{sim}/DM_count_{IDname}.npy')
cell_com = np.load(f'{sim}/cell_com_{IDname}.npy')
cell_gen = np.load(f'{sim}/cell_gen_{IDname}.npy')

print(fin_grid.shape)

Halo batch params (Rvir,Mvir,cNFW):
[[329.17613982  12.58066253   8.41010098]
 [298.72978142  12.45421305  10.25293176]]
2
1719691
(176, 1, 3)


In [3]:
print(fin_grid.shape, DM_count.shape, cell_com.shape, cell_gen.shape)
print(DM_count.sum() - len(DM_raw))
# note: 1 DM particle is "missing" in grid
print(f'snapshot {snap} : cell division rounds: {cell_division_count}')

(176, 1, 3) (176,) (176, 3) (176,)
-1
snapshot 0036 : cell division rounds: 3


In [ ]:
# --------------------------------------------- #
# Calculate gravity grid (in batches of cells). #
# --------------------------------------------- #

batch_size = 50
bs_cc = chunks(batch_size, fin_grid)
bs_count = chunks(batch_size, DM_count)
bs_com = chunks(batch_size, cell_com)
bs_gen = chunks(batch_size, cell_gen)

b_nums = []
for b, (b_cc,  b_gen,  b_com,  b_count) in enumerate(
    zip(bs_cc, bs_gen, bs_com, bs_count)
):
    b_nums.append(b)
    b_cc = np.array(b_cc)
    b_gen = np.array(b_gen)
    b_com = np.array(b_com)
    b_count = np.array(b_count)

    # Calculate gravity in each cell in current batch.
    b_DM = np.repeat(DM_pos, len(b_cc), axis=0)
    bname = f'batch{b}_Test'
    fct.cell_gravity(
        b_cc, b_com, b_gen, b_DM, b_count, 
        sim, bname
    )
bs_nums = np.array(b_nums)

# Combine and then delete batch files.
dPsi_batches = [
    np.load(f'{sim}/dPsi_grid_batch{b}_Test.npy') for b in bs_nums
]
dPsi_fin = np.array(list(chain.from_iterable(dPsi_batches)))
np.save(f'{sim}/dPsi_grid_{IDname}.npy', dPsi_fin)

# Delete intermediate data.
fct.delete_temp_data(f'{sim}/dPsi_*batch*Test.npy') 
fct.delete_temp_data(f'{sim}/DM_pos_*.npy')